In [1]:
import pandas as pd
from google.colab import drive


In [4]:
df = pd.read_csv(r'/content/drive/MyDrive/patent project/clean_data_with_enrich.csv')

<ipython-input-4-f9a6af3867a4>:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'/content/drive/MyDrive/patent project/clean_data_with_enrich.csv')


In [5]:
#calculating the reject rate for each uspsc class
df = df[df['Patent_Target']!= 'pending']

In [17]:
cat_data = df[['uspc_class','Patent_Target']]

In [19]:
cat_data['Patent_Target'] = cat_data['Patent_Target'].apply(lambda x: 0 if x == 'patented' else 1)

<ipython-input-19-2d863d79c712>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_data['Patent_Target'] = cat_data['Patent_Target'].apply(lambda x: 0 if x == 'patented' else 1)


In [20]:
cat_data['Patent_Target'].value_counts()

0    160264
1     70441
Name: Patent_Target, dtype: int64

In [26]:
cat_data = cat_data.groupby(['uspc_class'])['Patent_Target'].agg(['count','sum']).reset_index()

In [27]:
cat_data['Reject_Rate'] = round(cat_data['sum']/cat_data['count'],2)

In [28]:
cat_data = cat_data[['uspc_class','Reject_Rate']]

In [33]:
cat_data

,uspc_class,Reject_Rate
0,2,0.59
1,4,0.41
2,5,0.43
3,7,0.37
4,8,0.36
...,...,...
797,725,0.24
798,726,0.18
799,800,0.07
800,850,0.50


In [90]:
#top male categories:
top_male = df[df['one_if_male']==1]['uspc_class'].value_counts(normalize=True).head(25).reset_index().merge(cat_data,how='inner',left_on='index',right_on='uspc_class')

In [69]:
top_male.head()


,index,uspc_class_x,uspc_class_y,Reject_Rate
0,705,0.041054,705,0.71
1,709,0.016680,709,0.24
2,701,0.016265,701,0.18
3,340,0.015007,340,0.27
4,370,0.014641,370,0.14


In [91]:
top_male = top_male[['uspc_class_y','uspc_class_x'	,'Reject_Rate']]

In [59]:

def reject_rate(rate):
  if rate <= 0.5:
    return 'accepted'
  else:
    return 'reject'

def highlight_cells(val):
  if val ==  'accepted':
    color = 'green'
  elif val == 'reject':
    color = 'red'
  else:
    color = 'yellow'
  return 'background-color: {}'.format(color)


In [92]:
top_male.rename({'uspc_class_y':'USPC_Class','uspc_class_x':'Percentage','Reject_Rate':'Probable_Outcome'},axis=1,inplace=True)

In [93]:
top_male['Probable_Outcome'] = top_male['Probable_Outcome'].apply(reject_rate)

In [94]:
top_male.style.applymap(highlight_cells,subset=['Probable_Outcome'])

,USPC_Class,Percentage,Probable_Outcome
0,705,0.041054,reject
1,709,0.016680,accepted
2,701,0.016265,accepted
3,340,0.015007,accepted
4,370,0.014641,accepted
5,345,0.014453,accepted
6,800,0.014274,accepted
7,455,0.014048,accepted
8,707,0.013281,accepted
9,726,0.013069,accepted


In [95]:
top_female = df[df['one_if_male']==0]['uspc_class'].value_counts(normalize=True).head(25).reset_index().merge(cat_data,how='inner',left_on='index',right_on='uspc_class')

In [96]:
top_female = top_female[['uspc_class_y','uspc_class_x'	,'Reject_Rate']]

In [97]:
top_female.rename({'uspc_class_y':'USPC_Class','uspc_class_x':'Percentage','Reject_Rate':'Probable_Outcome'},axis=1,inplace=True)

In [98]:
top_female['Probable_Outcome'] = top_female['Probable_Outcome'].apply(reject_rate)

In [99]:
top_female.style.applymap(highlight_cells,subset=['Probable_Outcome'])

,USPC_Class,Percentage,Probable_Outcome
0,705,0.046627,reject
1,2,0.035668,reject
2,424,0.032230,reject
3,514,0.018393,accepted
4,119,0.018178,accepted
5,5,0.017834,accepted
6,132,0.016760,reject
7,206,0.016244,accepted
8,604,0.016244,accepted
9,340,0.016244,accepted
